# Limpieza y transformación de datos

En este notebook aplicamos las transformaciones necesarias detectadas durante el Análisis Exploratorio de Datos (EDA).

El objetivo es obtener un dataset limpio, coherente y listo para el análisis estadístico final.

---

## Resumen de decisiones tomadas tras el EDA

A partir del análisis previo, se han definido las siguientes acciones de limpieza:


### 1️⃣ Normalización de nombres de columnas

- Aplicar `strip()` y convertir los nombres a formato `snake_case`.
- Esto mejora la legibilidad y facilita el trabajo posterior con el dataset.
  

### 2️⃣ Variables categóricas

- No se han detectado inconsistencias en las categorías.
- Se aplicará únicamente `strip()` por seguridad para eliminar posibles espacios invisibles.
- No se modificarán mayúsculas/minúsculas ni se alterarán las categorías originales.
  

### 3️⃣ Variable `Salary`

- Existen valores negativos (0,12%), que son inconsistentes.
- Existen valores nulos concentrados en el grupo `Education = College`.

Decisión de tratamiento:
- Los valores negativos se tratarán como inválidos.
- Tanto los negativos como los nulos se imputarán usando la **mediana del grupo educativo correspondiente**.
- Para el grupo `College`, que no tiene salarios informados, se utilizará como referencia la mediana del grupo educativo más cercano.

El objetivo es no dejar valores nulos en `Salary` y aplicar una imputación coherente y robusta frente a outliers.


### 4️⃣ Cancelación

- Se crea una variable booleana `cancelled` (True/False) a partir de `Cancellation Year`:
  - `True` cuando existe año de cancelación.
  - `False` cuando no existe (valor faltante).

- Las columnas `cancellation_year` y `cancellation_month` se mantienen como valores faltantes (`<NA>`) cuando el cliente no ha cancelado.
  Estos valores no representan datos perdidos, sino un caso “no aplica”.

- Se convierten ambas columnas a tipo entero nullable (`Int64`) para evitar decimales innecesarios y mantener consistencia en el tipo de dato.

De esta forma, mantenemos la coherencia semántica de las variables originales y facilitamos su uso posterior en análisis estadístico o modelos predictivos.


### 5️⃣ Variables constantes

- `Country` tiene un único valor ("Canada").
- Se eliminará del dataset por no aportar información analítica.



In [3]:
# =========================================
# 1) Imports + carga del dataset unido (sucio)
# =========================================

"""
Cargamos el dataset unido (sin limpiar) que generamos en el notebook de merge.
Configuramos pandas para ver mejor resultados durante el proceso.
"""

import numpy as np
import pandas as pd
from IPython.display import display

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)

ruta_entrada = "../data/processed/dataset_unido.csv"
df = pd.read_csv(ruta_entrada)

print("✅ Dataset cargado")
print("Filas:", df.shape[0], "| Columnas:", df.shape[1])
display(df.head(3))


✅ Dataset cargado
Filas: 401688 | Columnas: 25


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
1,100018,2017,2,2,2,4,1320,132.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
2,100018,2017,3,14,3,17,2533,253.0,438,36,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN


In [4]:
# =========================================
# 2) Función de limpieza + guardado
# =========================================

def limpiar_dataset(df, ruta_salida="../data/processed/cleaned_customer_loyalty_flights.csv"):
    """
    Limpia y transforma el dataset unido siguiendo las decisiones tomadas tras el EDA.

    Acciones principales:
    1) Normaliza nombres de columnas a snake_case.
    2) Aplica strip() a variables categóricas (sin cambiar mayúsculas/minúsculas).
    3) Crea la variable booleana 'cancelled' y ajusta tipos de cancelación.
    4) Limpia Salary: negativos -> imputación + nulos -> imputación por mediana de Education.
       - Caso especial: Education='College' no tiene salarios -> fallback con mediana de Bachelor.
    5) Elimina 'Country' si es constante (un único valor).
    6) Guarda el dataset limpio en CSV.

    Nota:
    - No se eliminan outliers automáticamente.
    - Cancellation Year/Month se mantienen como NaN cuando no aplica.
    """

    df_limpio = df.copy()

    print("🧹 INICIO LIMPIEZA")
    print("-" * 60)
    print("Filas iniciales:", df_limpio.shape[0], "| Columnas iniciales:", df_limpio.shape[1])

    # -------------------------------------------------
    # 1) Normalizar nombres de columnas (snake_case)
    # -------------------------------------------------
    print("\n1) Normalizando nombres de columnas (snake_case)...")

    columnas_originales = df_limpio.columns.tolist()

    def a_snake_case(texto):
        """
        Convierte un texto a snake_case de forma simple:
        - strip
        - lower
        - reemplaza espacios por _
        """
        texto = texto.strip().lower()
        texto = texto.replace(" ", "_")
        return texto

    df_limpio.columns = [a_snake_case(col) for col in df_limpio.columns]

    print("✅ Columnas normalizadas. Ejemplos:")
    print("Antes:", columnas_originales[:5])
    print("Después:", df_limpio.columns.tolist()[:5])

    # -------------------------------------------------
    # 2) Strip en categóricas (por seguridad)
    # -------------------------------------------------
    print("\n2) Aplicando strip() en columnas categóricas...")

    columnas_categoricas = df_limpio.select_dtypes(include="object").columns.tolist()

    # Aplicamos strip solo a texto
    for col in columnas_categoricas:
        df_limpio[col] = df_limpio[col].astype(str).str.strip()

        # Al convertir a str, los NaN pasan a "nan".
        # En el EDA hemos comprobado que en el dataset las categóricas no tenían nulos, pero por seguridad:
        df_limpio[col] = df_limpio[col].replace("nan", np.nan)

    print("✅ Strip aplicado a:", len(columnas_categoricas), "columnas categóricas")

    # -------------------------------------------------
    # 3) Cancelación: crear cancelled + ajustar tipos
    # -------------------------------------------------
    print("\n3) Creando variable 'cancelled' y ajustando tipos de cancelación...")

    # Creamos booleano a partir de cancellation_year (NaN => no cancelado)
    df_limpio["cancelled"] = df_limpio["cancellation_year"].notna()

    # Convertimos year/month a Int64 nullable (mantiene NaN)
    df_limpio["cancellation_year"] = df_limpio["cancellation_year"].astype("Int64")
    df_limpio["cancellation_month"] = df_limpio["cancellation_month"].astype("Int64")

    print("✅ 'cancelled' creada. Distribución:")
    display(df_limpio["cancelled"].value_counts().to_frame(name="frecuencia"))

    # -------------------------------------------------
    # 4) Salary: negativos y nulos
    # -------------------------------------------------
    print("\n4) Limpieza e imputación de 'salary'...")

    # a) Contar negativos antes
    negativos_salary = (df_limpio["salary"] < 0).sum()
    nulos_salary = df_limpio["salary"].isna().sum()
    print("Salarios negativos detectados:", negativos_salary)
    print("Nulos en salary detectados:", nulos_salary)

    # b) Convertir negativos a NaN (tratarlos como inválidos)
    df_limpio.loc[df_limpio["salary"] < 0, "salary"] = np.nan

    # c) Medianas por Education (solo para grupos con datos)
    mediana_por_edu = df_limpio.groupby("education")["salary"].median()

    # d) Fallback para College (no tiene salarios): usar mediana de Bachelor si existe
    mediana_bachelor = mediana_por_edu.get("Bachelor", np.nan)

    if pd.isna(mediana_bachelor):
        # Si por algún motivo no existe Bachelor (muy raro), usamos mediana global
        mediana_bachelor = df_limpio["salary"].median()

    print("Mediana Salary (Bachelor) usada como fallback para College:", mediana_bachelor)

    # e) Imputación por Education:
    # - Si education != College: rellenar con mediana de su grupo
    # - Si education == College: rellenar con mediana_bachelor
    # Nota: hacemos esto en pasos simples para que sea fácil de entender.

    # Primero imputamos donde hay mediana definida por grupo
    df_limpio["salary"] = df_limpio["salary"].fillna(df_limpio["education"].map(mediana_por_edu))

    # Luego, lo que siga siendo NaN (principalmente College) lo rellenamos con fallback Bachelor
    df_limpio.loc[df_limpio["salary"].isna(), "salary"] = mediana_bachelor

    # Comprobación final salary
    print("✅ Salary tras imputación:")
    print("Nulos en salary:", df_limpio["salary"].isna().sum())
    print("Mínimo salary:", df_limpio["salary"].min())
    print("Máximo salary:", df_limpio["salary"].max())

    # -------------------------------------------------
    # 5) Eliminar Country si es constante
    # -------------------------------------------------
    print("\n5) Eliminando 'country' si es constante...")

    if "country" in df_limpio.columns:
        n_paises = df_limpio["country"].nunique(dropna=False)
        print("Valores únicos en country:", n_paises)

        if n_paises == 1:
            df_limpio = df_limpio.drop(columns=["country"])
            print("✅ 'country' eliminada (no aporta información).")
        else:
            print("ℹ️ 'country' se mantiene (tiene más de un valor).")

    # -------------------------------------------------
    # 6) Guardar CSV limpio
    # -------------------------------------------------
    print("\n6) Guardando dataset limpio...")
    df_limpio.to_csv(ruta_salida, index=False)

    print("✅ Guardado completado:", ruta_salida)
    print("Filas finales:", df_limpio.shape[0], "| Columnas finales:", df_limpio.shape[1])

    # Resumen final de nulos
    print("\n📌 Resumen final de nulos (top 10):")
    display(df_limpio.isna().sum().sort_values(ascending=False).head(10).to_frame(name="nulos"))

    return df_limpio

In [5]:
#Ejecución de la función de limpieza

df_limpio = limpiar_dataset(df)
display(df_limpio.head(3))

🧹 INICIO LIMPIEZA
------------------------------------------------------------
Filas iniciales: 401688 | Columnas iniciales: 25

1) Normalizando nombres de columnas (snake_case)...
✅ Columnas normalizadas. Ejemplos:
Antes: ['Loyalty Number', 'Year', 'Month', 'Flights Booked', 'Flights with Companions']
Después: ['loyalty_number', 'year', 'month', 'flights_booked', 'flights_with_companions']

2) Aplicando strip() en columnas categóricas...
✅ Strip aplicado a: 9 columnas categóricas

3) Creando variable 'cancelled' y ajustando tipos de cancelación...
✅ 'cancelled' creada. Distribución:


,frecuencia
cancelled,
False,352080
True,49608



4) Limpieza e imputación de 'salary'...
Salarios negativos detectados: 480
Nulos en salary detectados: 101712
Mediana Salary (Bachelor) usada como fallback para College: 72026.0
✅ Salary tras imputación:
Nulos en salary: 0
Mínimo salary: 15609.0
Máximo salary: 407228.0

5) Eliminando 'country' si es constante...
Valores únicos en country: 1
✅ 'country' eliminada (no aporta información).

6) Guardando dataset limpio...
✅ Guardado completado: ../data/processed/cleaned_customer_loyalty_flights.csv
Filas finales: 401688 | Columnas finales: 25

📌 Resumen final de nulos (top 10):


,nulos
cancellation_year,352080
cancellation_month,352080
month,0
year,0
flights_with_companions,0
total_flights,0
distance,0
flights_booked,0
loyalty_number,0
points_redeemed,0


,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month,cancelled
0,100018,2017,1,3,0,3,1521,152.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,<NA>,<NA>,False
1,100018,2017,2,2,2,4,1320,132.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,<NA>,<NA>,False
2,100018,2017,3,14,3,17,2533,253.0,438,36,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,<NA>,<NA>,False
